In [1]:
import numpy as np
from nanonets import Simulation

In [2]:
topo        = {"Nx" : 1, "Ny" : 1, "electrode_type" : None}
sim_class   = Simulation(topo)
sim_class.get_inv_capacitance_matrix()

array([[0.10580657]])